In [4]:
import csv
from web3 import Web3

In [5]:
# Alchemy API endpoint
alchemy_url = "https://eth-mainnet.g.alchemy.com/v2/D3TvA93qUD44YZgQu1ouV9upn3m-uIYP"  # Update with your Alchemy API endpoint

# Connect to the Alchemy API
w3 = Web3(Web3.HTTPProvider(alchemy_url))

In [10]:
# Ethereum block number to fetch
block_number = 11473329  # Ethereum blockchain, block number 11,473,329

# Fetch the block data from Alchemy
block = w3.eth.get_block(block_number)

# Define the CSV file name
csv_file_name = f"block_{block_number}_transactions.csv"

# Write the transaction data to a CSV file
with open(csv_file_name, mode="w", newline="", encoding="utf-8") as csv_file:
    fieldnames = [
        "transaction_hash",
        "from_address",
        "to_address",
        "value_wei",
        "gas",
        "gas_price_wei",
        "input_data",
        # Add more fields here
    ]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write the header
    writer.writeheader()

    # Extract and write transaction data
    for tx_hash in block.transactions:
        tx = w3.eth.get_transaction(tx_hash)
        tx_data = {
            "transaction_hash": tx.hash.hex(),
            "from_address": tx["from"],
            "to_address": tx["to"],
            "value_wei": tx["value"],
            "gas": tx["gas"],
            "gas_price_wei": tx["gasPrice"],
            "input_data": tx["input"],
            # Add more data fields here
        }
        writer.writerow(tx_data)

print(f"Transaction data saved to {csv_file_name}")

Transaction data saved to block_11473329_transactions.csv
